### Managing External Tables

#### version for local file

In [7]:
# imports

import pandas as pd

In [21]:
# constants and parameters
FILE_NAME = 'test_ratings.csv'

# This is the base_url for the region where the bucket is
BASE_URL = 'https://objectstorage.eu-frankfurt-1.oraclecloud.com/n/'

# the name for the external table
TABLE_NAME = 'RATINGS_EXT'

NAMESPACE = 'emXXXXXXX'
BUCKET_NAME = 'datalake_in'
CREDENTIAL_NAME = 'ADWH2'

# FILE_URL = BASE_URL + NAMESPACE + '/b/' + BUCKET_NAME + '/o/' + FILE_NAME

In [22]:
# This is the dictionary of supported mappings
dict_mappings = {}
dict_mappings['int32'] = 'NUMBER'
dict_mappings['int64'] = 'NUMBER'
dict_mappings['float32'] = 'NUMBER'
dict_mappings['float64'] = 'NUMBER'
dict_mappings['object'] = 'VARCHAR2(2000)'

dict_mappings

{'int32': 'NUMBER',
 'int64': 'NUMBER',
 'float32': 'NUMBER',
 'float64': 'NUMBER',
 'object': 'VARCHAR2(2000)'}

In [23]:
# read the file to link with external table

# for now, it assumes that the file is a plain CSV, separator is comma
my_df = pd.read_csv(FILE_NAME, sep=',')

# my_df.head()

# assumption made on file
# 1. It is a CSV file. Separator is comma
# 2. File is available locally (or on the Object Storage)
# 3. For now only supported number and strings (varchar2)
# 4. Header: mandatory, in first line
# ....

In [36]:
#
# function to generate ddl
#
def generate_ddl(df):
    '''
    df: Pandas DataFrame
    '''
    
    print('Generating DDL for table: ', TABLE_NAME)
    print()
    
    # const
    PART_INITIAL = "BEGIN\n \tDBMS_CLOUD.CREATE_EXTERNAL_TABLE("
    PART_END = "\t); \nEND; \n/"
    
    FORMAT_STRING = "format => json_object('type' value 'csv', 'skipheaders' value '1'),"
    FILE_URL = BASE_URL + NAMESPACE + '/b/' + BUCKET_NAME + '/o/' + FILE_NAME
    PART_TABLE_NAME = "table_name =>'" + TABLE_NAME + "',"
    PART_CREDENTIAL_NAME = "credential_name =>'" + CREDENTIAL_NAME + "',"
    PART_FILE_URI_LIST = "file_uri_list =>'" + FILE_URL + "',"
    
    # \t for pretty printing
    TAB2 = '\t\t'
    
    print(PART_INITIAL)
    print(TAB2 + PART_TABLE_NAME)
    print(TAB2 + PART_CREDENTIAL_NAME)
    print(TAB2 + PART_FILE_URI_LIST)
    print(TAB2 + FORMAT_STRING)
    
    # generate column list
    print(TAB2 + "column_list => '")
    
    # get list of columns
    l_columns = list(df.columns)
    
    # needed to handle last comma (not to be printed)
    n_columns = len(l_columns)
    COMMA = ","
    
    for i in range(n_columns):
        column = l_columns[i]
        
        # do not add last comma
        if i == n_columns - 1:
            COMMA = ""
        # column names changed to uppercase
        print(TAB2 + column.upper() + ' ' + dict_mappings[str(df[column].dtype)] + COMMA)
    
    # close column_list
    print(TAB2 + "'")
    
    # end of SQL
    print(PART_END)
    
    return

In [38]:
generate_ddl(my_df)

Generating DDL for table:  RATINGS_EXT

BEGIN
 	DBMS_CLOUD.CREATE_EXTERNAL_TABLE(
		table_name =>'RATINGS_EXT',
		credential_name =>'ADWH2',
		file_uri_list =>'https://objectstorage.YYYYYY.oraclecloud.com/n/XXXXX/b/datalake_in/o/test_ratings.csv',
		format => json_object('type' value 'csv', 'skipheaders' value '1'),
		column_list => '
		USER_ID NUMBER,
		MOVIE_ID NUMBER,
		RATING NUMBER,
		TS NUMBER
		'
	); 
END; 
/
